# Reranking Test Notebook

__Goals__
- Implement nqe
- Implement alpha query expansion
- Implement diffusion

____
## Imports

In [23]:
import numpy as np
import pandas as pd
import sklearn.metrics

____
## Query Expansion

In [2]:
def new_expanded_query(original_query_results, query_target_features, type = "n", n = 5):
    """Return an expanded query to based on initial query results."""
    
    top_n_feature_sets = query_target_features.loc[:n]

    if type == "n":
        pd.DataFrame([a.apply(np.mean)]).values

    if type == "alpha":
        pass

In [54]:
test_features = pd.DataFrame({1 : [1,2,3,4],
                              2 : [1,2,3,4],
                              3 : [1,2,3,4]})
results = [2,0,1,3]

n = 2

a = test_features.loc[results[:2]]
a

,1,2,3
2,3,3,3
0,1,1,1


In [83]:
b = pd.DataFrame([a.apply(np.mean)]).values
b

array([[2., 2., 2.]])

In [77]:
c = np.array(a.apply(np.mean))
type(c)

numpy.ndarray

In [78]:
type(b.values)

numpy.ndarray

In [81]:
np.shape(b.values)

(1, 3)

In [76]:
basic_query(c, test_features)

ValueError: Expected 2D array, got 1D array instead:
array=[2. 2. 2.].
Reshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample.

In [67]:
type(b.values)

numpy.ndarray

In [42]:
def basic_query(query_image_features, query_target_features, metric_function = sklearn.metrics.pairwise.euclidean_distances):
    """Return the indexes of the query_target images, arranged in ascending euclidean distance as compared to the query image"""
    
    query = query_image_features.reshape((1, -1))
    D = metric_function(query_target_features, query_image_features).squeeze()
    index = np.argsort(D)

    return(index)

In [48]:
a = test_features.loc[1].values.reshape((1,-1))
a

array([[2, 2, 2]])

In [50]:
D = sklearn.metrics.euclidean_distances(a, test_features).squeeze()
D

array([1.73205081, 0.        , 1.73205081, 3.46410162])

In [51]:
basic_query(a, test_features)

array([1, 0, 2, 3])